In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/GitHub/BERT_headline_classifier_v2')


Mounted at /content/drive


In [ ]:
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler
import numpy as np
from sklearn.model_selection import train_test_split

Mounted at /content/drive


In [ ]:
import os


In [ ]:
#import data
headlines_annotated = pd.read_csv("./data/headlines_annotated.tsv", sep="\t")

In [ ]:
#select only relevant columns
headlines_annotated = headlines_annotated[['headline', 'dominant_emotion']]

In [ ]:
headlines_annotated

,headline,dominant_emotion
0,Cops in One Village Have Been Convicted of 70 ...,anger
1,DIY penis enlargements are a 'nationwide probl...,negative_surprise
2,Dam breaking: New Epstein accuser comes forward,anger
3,David Beckham gets six-month driving ban for u...,negative_surprise
4,Dead sea turtle found with spear through head ...,sadness
...,...,...
4995,‘NOW do abortion. We’ll wait.’ Ilhan Omar doub...,negative_surprise
4996,‘Triad’ Thugs Use Clubs to Punish Hong Kong’s ...,negative_surprise
4997,'For real': High school football season begins...,positive_anticipation_including_optimism
4998,100 years on: the picture that changed our vie...,positive_surprise


In [ ]:
headlines_annotated["dominant_emotion"].value_counts()

dominant_emotion
negative_surprise                            893
positive_surprise                            582
anger                                        460
annoyance                                    426
fear                                         419
sadness                                      386
disgust                                      382
negative_anticipation_including_pessimism    323
positive_anticipation_including_optimism     319
joy                                          264
guilt                                        175
trust                                        124
shame                                        119
pride                                         65
love_including_like                           63
Name: count, dtype: int64

In [ ]:
#replace emotions with optimism and pessimism

replace_with_optimisim = ["positive_surprise", "positive_anticipation_including_optimism"]

replace_with_pessimism = ["negative_surprise", "negative_anticipation_including_pessimism"]


optimism_dict = {k: "optimism" for k in replace_with_optimisim}

pessimism_dict = {k: "pessimism" for k in replace_with_pessimism}

headlines_annotated["dominant_emotion"] = headlines_annotated["dominant_emotion"].replace(optimism_dict)
headlines_annotated["dominant_emotion"] = headlines_annotated["dominant_emotion"].replace(pessimism_dict)

headlines_annotated["dominant_emotion"].value_counts()

dominant_emotion
pessimism              1216
optimism                901
anger                   460
annoyance               426
fear                    419
sadness                 386
disgust                 382
joy                     264
guilt                   175
trust                   124
shame                   119
pride                    65
love_including_like      63
Name: count, dtype: int64

In [ ]:
#see value counts
headlines_annotated["dominant_emotion"].value_counts()

dominant_emotion
pessimism              1216
optimism                901
anger                   460
annoyance               426
fear                    419
sadness                 386
disgust                 382
joy                     264
guilt                   175
trust                   124
shame                   119
pride                    65
love_including_like      63
Name: count, dtype: int64

In [ ]:
#convert emotions besides pessimism and optimism into "other"
headlines_annotated.loc[~headlines_annotated["dominant_emotion"].isin(["optimism", "pessimism"]), "dominant_emotion"] = "other"

In [ ]:
#see value counts
headlines_annotated["dominant_emotion"].value_counts()

dominant_emotion
other        2883
pessimism    1216
optimism      901
Name: count, dtype: int64

In [ ]:
#undersample the majority class
rus = RandomUnderSampler(random_state=69)
X_resampled, y_resampled = rus.fit_resample(headlines_annotated["headline"].values.reshape(-1, 1), headlines_annotated["dominant_emotion"].values)


In [ ]:
#merge the two arrays back into pandas dataframe
headlines_resampled = pd.DataFrame(np.hstack((X_resampled, y_resampled.reshape(-1, 1))), columns=["headline", "dominant_emotion"])


In [ ]:
headlines_resampled

,headline,dominant_emotion
0,Elizabeth Warren Lays Out a Theory of Change a...,optimism
1,Emojis That Don’t Exist but Should,optimism
2,Enbridge Is Behind This Front Group Pushing th...,optimism
3,Europe’s Dream: Escaping the Dictatorship of t...,optimism
4,"Getting 'California Sober' Showed Me a Kinder,...",optimism
...,...,...
2698,Rubio says progressive Democrats are 'fully' A...,pessimism
2699,"Segregation Is Back, and It’s Coming From the ...",pessimism
2700,Donald Trump goes completely off the deep end ...,pessimism
2701,"I Make $1,000 a Week Writing Essays for Lazy S...",pessimism


In [ ]:
#see value counts
headlines_resampled["dominant_emotion"].value_counts()

dominant_emotion
optimism     901
other        901
pessimism    901
Name: count, dtype: int64

In [ ]:
#checking the length of X
len(X_resampled)

2703

In [ ]:
#split into training, testing, and validation sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=69)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=69)

In [ ]:
#merge the X and ys together into pandas dataframe
train = pd.DataFrame(np.hstack((X_train, y_train.reshape(-1, 1))), columns=["headline", "dominant_emotion"])
test = pd.DataFrame(np.hstack((X_test, y_test.reshape(-1, 1))), columns=["headline", "dominant_emotion"])
val = pd.DataFrame(np.hstack((X_val, y_val.reshape(-1, 1))), columns=["headline", "dominant_emotion"])


In [34]:
#export to csv
train.to_csv("./data/train.csv", index=False)
test.to_csv("./data/test.csv", index=False)
val.to_csv("./data/val.csv", index=False)
